In [27]:
import logging as log
import numpy as np
from ortools.linear_solver import pywraplp

### Set up logging


In [28]:
for handler in log.root.handlers[:]:
    log.root.removeHandler(handler)
log.basicConfig(format="%(levelname)s - %(message)s")

logger = log.getLogger()
logger.setLevel(log.DEBUG)
log.info("Logger initialized...")

INFO - Logger initialized...


### Get input


In [29]:
def getinput(file):
    with open(file) as data:
        NumNode = int(next(data))
        PathCost = np.loadtxt(data)
    return NumNode, PathCost

In [30]:
NumNode, PathCost = getinput("largeTSP.txt")

### Set up solver


In [31]:
solver_id = "SCIP"
solver: pywraplp.Solver = pywraplp.Solver.CreateSolver(solver_id)
inf = solver.infinity()

### Generating decision variables


In [32]:
log.info("Creating " + str(NumNode * NumNode) + " boolean x_ij variables... ")
X = {}
for i in range(NumNode):
    for j in range(NumNode):
        X[(i, j)] = solver.BoolVar(f"x{(i, j)}")

INFO - Creating 900 boolean x_ij variables... 


### Creating constraints


In [33]:
# Constraint 1: leave every point exactly once

log.info("Creating " + str(NumNode - 1) + " Constraint 1... ")
for i in range(NumNode):
    solver.Add(sum(X[(i, j)] for j in range(NumNode) if j!=i) == 1)

INFO - Creating 29 Constraint 1... 


In [34]:
# Constraint 2: reach every point from exactly one other point

log.info("Creating " + str(NumNode - 1) + " Constraint 2... ")
for j in range(NumNode):
    solver.Add(sum(X[(i, j)] for i in range(NumNode) if i!=j) == 1)

INFO - Creating 29 Constraint 2... 


### Dynamically remove subtours

In [35]:
def SubTourDetector(X):
    path = {start: end for start in range(NumNode) for end in range(NumNode) if X[(start, end)].solution_value() == 1} 
    Nodes = set(range(NumNode))
    subTours = [[]]
    start = 0
    numSubTour = 1
    while True:
        Nodes.remove(start)
        subTours[-1].append((start, path[start]))
        if Nodes:
            if path[start] in Nodes:
                start = path[start]
            else:
                start = next(iter(Nodes))
                subTours.append([])
                numSubTour += 1
        else:
            break
    return numSubTour, subTours


In [36]:
def removeSubTours(solver, X, subTours):
    for tour in subTours:
        log.info("Removing subtour")
        solver.Add(sum(X[start_end] for start_end in tour) <= len(tour) - 1)

### Define objective function


In [37]:
# Create objective function
solver.Minimize(
    sum(X[(i, j)] * PathCost[i][j] 
    for i in range(NumNode) 
    for j in range(NumNode))
)

### Print the solution

In [38]:
def printSolution(tour):
    start = 0
    print("Optimal tour:", start + 1, end = " ")
    for start, end in tour:
        print("->", end + 1, end=" ")
    print()

### Solve the model


In [39]:
log.info("Solving MIP model... ")
while True:
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        numSubTour, subTours = SubTourDetector(X)   
        if numSubTour == 1:
            print("Optimal objective value ==", solver.Objective().Value())
            break
        else:
            removeSubTours(solver, X, subTours)
    else:
        print("UNBOUNDED")
        break

printSolution(solTour := subTours[0])

INFO - Solving MIP model... 
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour
INFO - Removing subtour


Optimal objective value == 43398.999999999956
Optimal tour: 1 -> 16 -> 2 -> 11 -> 17 -> 12 -> 13 -> 10 -> 28 -> 4 -> 26 -> 20 -> 8 -> 6 -> 19 -> 5 -> 9 -> 22 -> 7 -> 24 -> 21 -> 18 -> 27 -> 29 -> 14 -> 30 -> 23 -> 25 -> 15 -> 3 -> 1 
